In [ ]:
import os
import copy
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Config():
    def __init__(self):
        self.use_horizontal_flip = True
        self.add_noise = True
        self.rotate_90 = True
        
        self.classes_name = ['bird', 'eagle', 'flamingo', 'golden oriole', 'hyacinth macaw', 'mute swan', 'seagull']
        self.int_name = {'bird': 0, 'eagle': 1, 'flamingo': 2, 'golden oriole': 3, 'hyacinth macaw': 4, 'mute swan': 5, 'seagull': 6}

In [ ]:
# Parser the data from drive

def get_data(filepath, C):
    all_imgs = {}
    image_path = os.path.join(filepath, 'images')
    label_path = os.path.join(filepath, 'labels')

    for files in os.listdir(image_path):
        all_imgs[files] = {}

        img = cv2.imread(os.path.join(image_path, files))
        
        all_imgs[files]['filename'] = files
        all_imgs[files]['height'] = img.shape[0]
        all_imgs[files]['width'] = img.shape[1]
        all_imgs[files]['bboxes'] = []

        filename = files[:-3] + 'txt' if files[-4:].lower() != 'jpeg' else files[:-4] + 'txt'

        with open(os.path.join(label_path, filename), 'r') as f:
            for line in f:
                s_line = [float(x) for x in line.split(' ')]
                all_imgs[files]['bboxes'].append({'class': C.classes_name[int(s_line[0])], 'x': s_line[1], 'y': s_line[2], 'w': s_line[3], 'h': s_line[4]})
    
    all_data = []

    for key in all_imgs:
        all_data.append(all_imgs[key])
    
    return all_data

In [ ]:
C = Config()
C.use_horizontal_flip = True
C.add_noise = True
C.rotate_90 = True

filepath = '/content/birds_data_part2'
all_imgs = get_data(filepath, C)

for img in all_imgs:
    print(img)

In [ ]:
print(len(all_imgs))
print(all_imgs[0])

In [ ]:
# Display image

def display_image(filepath, img_data, img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    for bbox in img_data['bboxes']:
        x1 = (bbox['x'] - bbox['w'] / 2) * img_data['width']
        x2 = (bbox['x'] + bbox['w'] / 2) * img_data['width']
        y1 = (bbox['y'] - bbox['h'] / 2) * img_data['height']
        y2 = (bbox['y'] + bbox['h'] / 2) * img_data['height']

        x1, x2, y1, y2 = int(x1), int(x2), int(y1), int(y2)
        
        color_green = (0, 255, 0)

        cv2.rectangle(img, (x1, y1), (x2, y2), color_green, 2)

    plt.figure(figsize = (10, 10))
    plt.grid()
    plt.imshow(img)
    plt.show()

In [ ]:
# Augment the image

def augment_image(filepath, img_data, C, augment = True):
    img_aug = []

    if augment:
        # Flipping
        if C.use_horizontal_flip:
            img_data_aug = copy.deepcopy(img_data)

            if img_data_aug['filename'][-4:].lower() != 'jpeg':
                img_data_aug['filename'] = img_data_aug['filename'][:-4] + '-flip' + img_data_aug['filename'][-4:]
            else:
                img_data_aug['filename'] = img_data_aug['filename'][:-5] + '-flip' + img_data_aug['filename'][-5:]

            img = cv2.imread(os.path.join(filepath, img_data_aug['filename']))
            img_debug = cv2.flip(img, 1)

            for bbox in img_data_aug['bboxes']:
                bbox['x'] = 1.0 - bbox['x']

            img_aug.append([img_data_aug, img_debug])

        # Adding noise
        if C.add_noise:
            img_data_aug = copy.deepcopy(img_data)

            if img_data_aug['filename'][-4:].lower() != 'jpeg':
                img_data_aug['filename'] = img_data_aug['filename'][:-4] + '-noise' + img_data_aug['filename'][-4:]
            else:
                img_data_aug['filename'] = img_data_aug['filename'][:-5] + '-noise' + img_data_aug['filename'][-5:]
            
            img = cv2.imread(os.path.join(filepath, img_data_aug['filename']))

            noise_type = np.random.choice(['gauss', 'salt-pepper', 'poisson', 'speckle'], 1)[0]

            # Gauss
            if noise_type == 'gauss':
                row, col, _ = img.shape
                mean = 0
                var = 10
                sigma = var ** 0.5

                gauss = np.random.normal(mean, sigma, (row, col))

                img_debug = np.zeros(img.shape, np.float32)
                img_debug[:, :, 0] = img[:, :, 0] + gauss
                img_debug[:, :, 1] = img[:, :, 1] + gauss
                img_debug[:, :, 2] = img[:, :, 2] + gauss

                cv2.normalize(img_debug, img_debug, 0, 255, cv2.NORM_MINMAX, dtype=-1)
                img_debug = img_debug.astype(np.uint8)

            # Salt & Pepper
            elif noise_type == 'salt-pepper':
                row, col, ch = img.shape
                s_vs_p = 0.5
                amount = 0.02
                img_debug = np.copy(img)

                # Salt mode
                num_salt = np.ceil(amount * img.size * s_vs_p)
                coords = [np.random.randint(0, i - 1, int(num_salt)) for i in img.shape]
                img_debug[coords] = 1
 
                # Pepper mode
                num_pepper = np.ceil(amount * img.size * (1.0 - s_vs_p))
                coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in img.shape]
                img_debug[coords] = 0

            # Poisson
            elif noise_type == 'poisson':
                vals = len(np.unique(img))
                vals = 2 ** np.ceil(np.log2(vals))
                img_debug = np.random.poisson(img / 255.0 * vals) / vals * 255

            # Speckle
            elif noise_type == 'speckle':
                row, col, ch = img.shape
                gauss = np.random.randn(row, col, ch)
                gauss = gauss.reshape(row, col, ch).astype(np.uint8)    
                img_debug = img + img * gauss

            img_aug.append([img_data_aug, img_debug])

        # Rotating
        if C.rotate_90:
            img_data_aug = copy.deepcopy(img_data)

            if img_data_aug['filename'][-4:].lower() != 'jpeg':
                img_data_aug['filename'] = img_data_aug['filename'][:-4] + '-rotate' + img_data_aug['filename'][-4:]
            else:
                img_data_aug['filename'] = img_data_aug['filename'][:-5] + '-rotate' + img_data_aug['filename'][-5:]
            
            img = cv2.imread(os.path.join(filepath, img_data_aug['filename']))

            angle = np.random.choice([90, 270], 1)[0]

            if angle == 90:
                img = np.transpose(img, (1, 0, 2))
                img_debug = cv2.flip(img, 1)

            elif angle == 270:
                img = np.transpose(img, (1, 0, 2))
                img_debug = cv2.flip(img, 0)

            img_data_aug['height'], img_data_aug['width'] = img_data_aug['width'], img_data_aug['height']
            
            for bbox in img_data_aug['bboxes']:
                x = bbox['x']
                y = bbox['y']
                w = bbox['w']
                h = bbox['h']

                bbox['w'] = h
                bbox['h'] = w

                if angle == 90:
                    bbox['x'] = 1.0 - y
                    bbox['y'] = x

                elif angle == 270:
                    bbox['x'] = y
                    bbox['y'] = 1.0 - x

            img_aug.append([img_data_aug, img_debug])
    
    return img_aug

In [ ]:
# Copy images to augment

image_old_path = '/content/birds_data_part2/images'
image_new_path = '/content/birds_data_part3/images'

for img in os.listdir(image_old_path):
    old_image = os.path.join(image_old_path, img)
    
    filename = img[:-4] + '-flip' + img[-4:] if img[-4:].lower() != 'jpeg' else img[:-5] + '-flip' + img[-5:]
    new_image = os.path.join(image_new_path, filename)
    shutil.copy(old_image, new_image)

    filename = img[:-4] + '-noise' + img[-4:] if img[-4:].lower() != 'jpeg' else img[:-5] + '-noise' + img[-5:]
    new_image = os.path.join(image_new_path, filename)
    shutil.copy(old_image, new_image)

    filename = img[:-4] + '-rotate' + img[-4:] if img[-4:].lower() != 'jpeg' else img[:-5] + '-rotate' + img[-5:]
    new_image = os.path.join(image_new_path, filename)
    shutil.copy(old_image, new_image)

In [ ]:
# Creat txt files

image_path = '/content/birds_data_part3/images'
label_path = '/content/birds_data_part3/labels'

for img_data in all_imgs:
    img_aug = augment_image(image_path, img_data, C)

    for img_data_aug, img_debug in img_aug:
        cv2.imwrite(os.path.join(image_path, img_data_aug['filename']), img_debug)

        txt_file = img_data_aug['filename'][:-3] + 'txt' if img_data_aug['filename'][-4:].lower() != 'jpeg' else img_data_aug['filename'][:-4] + 'txt'
        with open(os.path.join(label_path, txt_file), 'w') as f:
            for label in img_data_aug['bboxes']:
                label['class'] = C.int_name[label['class']]
                f.write('{} {} {} {} {}\n'.format(label['class'], label['x'], label['y'], label['w'], label['h']))